# Multiple Input and Output Channels (7.4)

Assume an input of shape ci × h × w and a convolution kernel of shape co × ci × kh × kw,
padding of (ph, pw), and stride of (sh, sw).
1. What is the computational cost (multiplications and additions) for the forward propagation?
2. What is the memory footprint?
3. What is the memory footprint for the backward computation?
4. What is the computational cost for the backpropagation?

In [1]:
#!pip install d2l==1.0.3

# d2l importing
from google.colab import drive
drive.mount('/content/gdrive')
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks')

# libraries needed
import torch
from torch import nn
from torch.nn import functional as F
import d2l
from d2l import torch as d2l

# Parameters
ci, h, w = 3, 32, 32  # Example input shape
co, kh, kw = 64, 3, 3  # Example convolution kernel shape
ph, pw = 1, 1  # Example padding
sh, sw = 1, 1  # Example stride

byteSize = 4 # for Float 32 memory

Mounted at /content/gdrive


#1 Forward Computational Cost

In [2]:
# Forward propagation computational cost (multiplications and additions)
# accounting for stride and padding
h_out = (h + 2 * ph - kh) // sh + 1
w_out = (w + 2 * pw - kw) // sw + 1

forward_mul = co * ci * kh * kw * h * w
forward_add = forward_mul * (kh * kw - 1)  # Assuming one addition per multiplication


print("Forward propagation computational cost (multiplications):", forward_mul)
print("Forward propagation computational cost (additions):", forward_add)
print("Forward propagation computational cost (total operations):", forward_add+forward_mul)

Forward propagation computational cost (multiplications): 1769472
Forward propagation computational cost (additions): 14155776
Forward propagation computational cost (total operations): 15925248


# 2  Memory Footprint



In [3]:
# Memory footprint for forward propagation
forward_memory = (co * h * w + ci * kh * kw) * byteSize  # Assuming 4 bytes for float32
print("Memory footprint for forward propagation:", forward_memory)

Memory footprint for forward propagation: 262252


#3 Backward Computation Memory

In [4]:
backward_memory = (co * h_out * w_out + ci * kh * kw) * byteSize * 2  # Assuming gradients for weights and inputs
print("Memory footprint for backward computation:", backward_memory)

Memory footprint for backward computation: 524504


# 4 Backward Computational Cost



In [5]:
# Backpropagation computational cost (multiplications and additions)
backward_mul = forward_mul * 2  # Gradients w.r.t. inputs and weights
backward_add = forward_add * 2  # Gradients w.r.t. inputs and weights

print("Backpropagation computational cost (multiplications):", backward_mul)
print("Backpropagation computational cost (additions):", backward_add)

Backpropagation computational cost (multiplications): 3538944
Backpropagation computational cost (additions): 28311552


# Other way test

In [6]:
# # Parameters
# ci, h, w = 3, 32, 32  # Example input shape
# co, kh, kw = 64, 3, 3  # Example convolution kernel shape
# ph, pw = 1, 1  # Example padding
# sh, sw = 1, 1  # Example stride
# byteSize = 4 # for Float 32 memory

# # Input tensor
# x = torch.randn(1, ci, h, w)

# # Convolutional layer
# conv = torch.nn.Conv2d(ci, co, kernel_size=(kh, kw), padding=(ph, pw), stride=(sh, sw))

# # Forward pass
# out = conv(x)

# # Backward pass
# loss = torch.randn_like(out).sum()  # Example loss
# loss.backward()

# # Memory footprint calculation
# memory_footprint = sum(p.numel() * byteSize for p in conv.parameters())  # Parameters memory
# memory_footprint += sum(t.numel() * byteSize for t in [x, out, loss.grad_fn.next_functions[0][0].variable] if t is not None)  # Activations and gradients memory

# # Number of additions and multiplications calculation
# flops_forward = (2 * ci * kh * kw * co * (h * w) - ci * co * (h * w))  # Forward pass FLOPs
# flops_backward = (2 * ci * kh * kw * co * (h * w) - ci * co * (h * w)) * 2  # Backward pass FLOPs

# print("Memory footprint (bytes):", memory_footprint)
# print("Number of additions (FLOPs) for forward pass:", flops_forward)
# print("Number of additions (FLOPs) for backward pass:", flops_backward)


RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn